# Overview

This is the final script of the final project for Civic Analytics at NYC CUSP. It includes the following sections:

- Data Ingestion and Processing
- Modelling
- Data Exporting
- Analysis and Visualization

Contributor
- Ian Xiao
- Ben Steers
- Te Du

---

## Data Ingestion and Processing

In [17]:
# Import
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.cluster import KMeans
from sklearn import preprocessing
import warnings
import matplotlib.style as style
import statsmodels.api as sm
import statsmodels.formula.api as smf
from IPython.display import HTML, display as d
from IPython.core.magic import register_cell_magic
from datetime import datetime
warnings.filterwarnings('ignore')
style.use('fivethirtyeight')
%pylab inline
pd.set_option('display.max_columns', 500)

Populating the interactive namespace from numpy and matplotlib


In [30]:
# Ingest data from project github
# Please see Data Processing script for detailed steps (TO BE UPLOADED)
url = "https://raw.githubusercontent.com/td928/CAUI_team1_Final_Project/master/merged-w-latlon.csv"
df = pd.read_csv(url)

In [42]:
# Create a Age feature based on Built Year
df["age"] = datetime.datetime.now().year - df['YearBuilt']

# Create Average EUI of 2013, 2014, and 2015
df['EUI_2013'] = pd.to_numeric(df['EUI_2013'], errors='coerce')
df['EUI_2014'] = pd.to_numeric(df['EUI_2014'], errors='coerce')
df['EUI_2015'] = pd.to_numeric(df['EUI_2015'], errors='coerce')
df["avgEUI"] = (df['EUI_2013']+df['EUI_2014']+df['EUI_2015'])/3

df.head(10)

,BBL,EUI_2013,floorArea_2013,EUI_2014,floorArea_2014,EUI_2015,floorArea_2015,EUI_2016,floorArea_2016,TYPE_2016,Borough,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,NumFloors,UnitsRes,UnitsTotal,YearBuilt,XCoord,YCoord,ZipCode,Address,Latitude,Longitude,age,avgEUI
0,1000010010,NaN,NaN,NaN,2725731.0,NaN,2725731.0,NaN,NaN,NaN,MN,7736692.0,2725731.0,2725731.0,0.0,0.0,0.0,0.0,0.0,0.0,1900.0,979071.0,190225.0,10004.0,1 GOVERNORS ISLAND,40.688799,-74.018675,117.0,NaN
1,1000020002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MN,191502.0,158197.0,158197.0,0.0,0.0,0.0,5.0,0.0,0.0,1900.0,981037.0,194506.0,10004.0,10 SOUTH STREET,40.700550,-74.011588,117.0,NaN
2,1000047501,102.0,2428325.0,92.5,2621563.0,149.0,2542563.0,133.8,2542563.0,Office,MN,111382.0,1888126.0,1888126.0,0.0,1848626.0,39500.0,50.0,0.0,52.0,1969.0,980917.0,195090.0,10004.0,1 WATER STREET,40.702153,-74.012021,48.0,114.500000
3,1000057501,119.6,1338000.0,134.5,1354691.0,113.8,1354691.0,135.8,1354691.0,Office,MN,54023.0,1024229.0,1024229.0,0.0,1024229.0,0.0,40.0,0.0,7.0,1970.0,981309.0,195131.0,10004.0,125 BROAD STREET,40.702266,-74.010607,47.0,122.633333
4,1003620001,30.3,5207812.0,NaN,458591.0,NaN,458591.0,NaN,NaN,See Primary BBL,MN,258600.0,458591.0,620.0,457971.0,0.0,620.0,6.0,586.0,590.0,1948.0,991053.0,202591.0,10009.0,134 AVENUE D,40.722740,-73.975457,69.0,NaN
5,1003620001,30.3,5207812.0,NaN,458591.0,NaN,458591.0,NaN,NaN,See Primary BBL,MN,258600.0,458591.0,620.0,457971.0,0.0,620.0,6.0,586.0,590.0,1948.0,991053.0,202591.0,10009.0,134 AVENUE D,40.722740,-73.975457,69.0,NaN
6,1000090001,99.5,852840.0,78.5,845018.0,81.3,845018.0,71.4,845018.0,Office,MN,29481.0,845018.0,845018.0,0.0,839651.0,5367.0,32.0,0.0,9.0,1970.0,980562.0,195311.0,10004.0,34 WHITEHALL STREET,40.702760,-74.013301,47.0,86.433333
7,1000090014,79.7,574095.0,68.6,544015.0,80.8,544015.0,77.2,544015.0,Office,MN,23080.0,544015.0,544015.0,0.0,542515.0,0.0,42.0,0.0,0.0,1987.0,980402.0,195359.0,10004.0,17 STATE STREET,40.702891,-74.013878,30.0,76.366667
8,1000090029,139.4,859807.0,148.4,896956.0,136.2,896956.0,118.6,896956.0,Office,MN,39985.0,896956.0,896956.0,0.0,864078.0,2878.0,35.0,0.0,9.0,1971.0,980443.0,195525.0,10004.0,24 WHITEHALL STREET,40.703347,-74.013730,46.0,141.333333
9,1000100014,134.5,405310.0,106.1,365792.0,113.0,365792.0,NaN,NaN,NaN,MN,13214.0,365792.0,365792.0,0.0,365792.0,0.0,30.0,0.0,15.0,1986.0,980734.0,195538.0,10004.0,33 WHITEHALL STREET,40.703383,-74.012681,31.0,117.866667


In [43]:
# Drop rows with any missing value
model_df = df.dropna()

print("--- Number of Missing Value in Each Column ---")
model_df.isnull().sum()

--- Number of Missing Value in Each Column ---


BBL               0
EUI_2013          0
floorArea_2013    0
EUI_2014          0
floorArea_2014    0
EUI_2015          0
floorArea_2015    0
EUI_2016          0
floorArea_2016    0
TYPE_2016         0
Borough           0
LotArea           0
BldgArea          0
ComArea           0
ResArea           0
OfficeArea        0
RetailArea        0
NumFloors         0
UnitsRes          0
UnitsTotal        0
YearBuilt         0
XCoord            0
YCoord            0
ZipCode           0
Address           0
Latitude          0
Longitude         0
age               0
avgEUI            0
dtype: int64

In [44]:
# Select only Multi-family buildings for Modelling 

model_df = model_df[(model_df.TYPE_2016 == 'Multifamily Housing')]
print("Number of Records in the Modelling Dataset: {}".format(model_df.shape[0]))

Number of Records in the Modelling Dataset: 7649


--- 

## Modelling
- Calculate EER using K-Means
- Calculate EER using Linear Regression

EER Calculation:
- In K-Means, EER = actual EUI 2016 / K-mean Group EUI 2016.
- In Linear Regression, EER = actual EUI 2016 / Predicted EUI 2016

In [45]:
model_df.dtypes

BBL                 int64
EUI_2013          float64
floorArea_2013    float64
EUI_2014          float64
floorArea_2014    float64
EUI_2015          float64
floorArea_2015    float64
EUI_2016          float64
floorArea_2016    float64
TYPE_2016          object
Borough            object
LotArea           float64
BldgArea          float64
ComArea           float64
ResArea           float64
OfficeArea        float64
RetailArea        float64
NumFloors         float64
UnitsRes          float64
UnitsTotal        float64
YearBuilt         float64
XCoord            float64
YCoord            float64
ZipCode           float64
Address            object
Latitude          float64
Longitude         float64
age               float64
avgEUI            float64
dtype: object

In [46]:
# Perform K-Means

kmean_feature = ['age', 'Latitude', 'Longitude', 'BldgArea', 'UnitsTotal', 'avgEUI']
init = 100
seed = 123

kmeans = KMeans(n_clusters=3, random_state = seed, n_init = init)

kmeans.fit(model_df[kmean_feature])
model_df['km_group'] = kmeans.predict(model_df[kmean_feature])

# @IAN updat

In [47]:
model_df.groupby('km_group').size()

km_group
0     855
1    6636
2     158
dtype: int64

In [50]:
# Calculate k-mean based EUI stats from Iteration 1
km_stats = pd.pivot_table(model_df, index= 'km_group', values= "EUI_2016",
                               aggfunc= [np.mean, np.median, min, max, np.std, np.size])

# Rename Columns by Joining with "_"
km_stats.columns = ["Group_"+"_".join((i,j)) for i,j in km_stats.columns]
km_stats = km_stats.reset_index()

km_stats

,km_group,Group_mean_EUI_2016,Group_median_EUI_2016,Group_min_EUI_2016,Group_max_EUI_2016,Group_std_EUI_2016,Group_size_EUI_2016
0,0,265.500117,105.1,0.0,49924.0,2026.773072,855.0
1,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
2,2,2944.470253,223.0,18.7,32513.5,7810.753742,158.0


In [51]:
# Merge with Original Modelling Dataset

km_df = model_df.merge(km_stats, how = "left", left_on = "km_group", right_on = "km_group")

In [53]:
km_df.sample(10)

,BBL,EUI_2013,floorArea_2013,EUI_2014,floorArea_2014,EUI_2015,floorArea_2015,EUI_2016,floorArea_2016,TYPE_2016,Borough,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,NumFloors,UnitsRes,UnitsTotal,YearBuilt,XCoord,YCoord,ZipCode,Address,Latitude,Longitude,age,avgEUI,km_group,Group_mean_EUI_2016,Group_median_EUI_2016,Group_min_EUI_2016,Group_max_EUI_2016,Group_std_EUI_2016,Group_size_EUI_2016
6185,4051820074,109.9,90754.0,108.2,87700.0,97.9,87700.0,75.7,87700.0,Multifamily Housing,QN,20600.0,87700.0,0.0,87700.0,0.0,0.0,6.0,84.0,84.0,1936.0,1033069.0,214489.0,11355.0,140-11 ASH AVENUE,40.755264,-73.823790,81.0,105.333333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
950,1010487501,73.2,237468.0,113.0,266115.0,192.8,266115.0,120.1,266115.0,Multifamily Housing,MN,12251.0,224336.0,2120.0,222216.0,2120.0,0.0,45.0,261.0,262.0,1982.0,988523.0,219066.0,10019.0,347 WEST 57 STREET,40.767961,-73.984574,35.0,126.333333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
7231,4076720002,296.5,1600.0,887.4,174200.0,112.3,174200.0,116.2,174200.0,Multifamily Housing,QN,372000.0,174200.0,0.0,174200.0,0.0,0.0,2.0,224.0,224.0,1949.0,1053453.0,210973.0,11364.0,224-01 69 AVENUE,40.745478,-73.750252,68.0,432.066667,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
6819,4071170006,48.9,16000.0,235.6,410788.0,305.5,410788.0,275.6,410788.0,Multifamily Housing,QN,790500.0,410788.0,24360.0,386428.0,0.0,0.0,2.0,410.0,411.0,1950.0,1045694.0,208925.0,11365.0,194-05 67 AVENUE,40.739914,-73.778272,67.0,196.666667,0,265.500117,105.1,0.0,49924.0,2026.773072,855.0
423,1006400021,73.0,71709.0,80.2,76549.0,81.0,76549.0,73.2,76549.0,Multifamily Housing,MN,12377.0,76549.0,8800.0,67749.0,0.0,0.0,11.0,67.0,67.0,2003.0,981904.0,207871.0,10014.0,756 WASHINGTON STREET,40.737234,-74.008465,14.0,78.066667,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
232,1005280001,114.5,71480.0,127.8,71490.0,26.4,71490.0,26.7,71490.0,Multifamily Housing,MN,9012.0,71490.0,0.0,71490.0,0.0,0.0,13.0,115.0,115.0,1952.0,983254.0,204733.0,10014.0,182 WEST HOUSTON STREET,40.728621,-74.003594,65.0,89.566667,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
1574,1013960071,73.5,95603.0,88.9,95603.0,91.2,95603.0,85.5,95603.0,Multifamily Housing,MN,10165.0,95603.0,1000.0,94603.0,1000.0,0.0,12.0,23.0,25.0,1913.0,993072.0,217786.0,10065.0,553 PARK AVENUE,40.764444,-73.968153,104.0,84.533333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
1836,1014990025,108.4,141049.0,121.4,156729.0,138.0,156729.0,121.8,156729.0,Multifamily Housing,MN,17900.0,156729.0,4320.0,152409.0,4320.0,0.0,14.0,120.0,129.0,1963.0,996037.0,223968.0,10128.0,55 EAST 87 STREET,40.781409,-73.957439,54.0,122.600000,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
1945,1015300034,94.0,55280.0,99.8,50853.0,106.4,50853.0,99.1,50853.0,Multifamily Housing,MN,7662.0,50853.0,0.0,50852.0,0.0,0.0,14.0,55.0,55.0,1946.0,997204.0,222493.0,10028.0,230 EAST 85 STREET,40.777359,-73.953228,71.0,100.066667,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0
1467,1013650009,77.0,430983.0,89.2,413233.0,92.5,413233.0,83.2,413233.0,Multifamily Housing,MN,28308.0,413233.0,36722.0,376511.0,0.0,272.0,37.0,323.0,325.0,1982.0,994358.0,214622.0,10022.0,420 EAST 54 STREET,40.755759,-73.963515,35.0,86.233333,0,265.500117,105.1,0.0,49924.0,2026.773072,855.0


In [54]:
km_df['EER_km'] = km_df['EUI_2016'] / km_df['Group_mean_EUI_2016']
km_df['EER_km'].describe()

count    7649.000000
mean        1.000000
std        12.620905
min         0.000000
25%         0.320431
50%         0.400892
75%         0.491704
max       547.033440
Name: EER_km, dtype: float64

In [56]:
km_df.sample(10)

,BBL,EUI_2013,floorArea_2013,EUI_2014,floorArea_2014,EUI_2015,floorArea_2015,EUI_2016,floorArea_2016,TYPE_2016,Borough,LotArea,BldgArea,ComArea,ResArea,OfficeArea,RetailArea,NumFloors,UnitsRes,UnitsTotal,YearBuilt,XCoord,YCoord,ZipCode,Address,Latitude,Longitude,age,avgEUI,km_group,Group_mean_EUI_2016,Group_median_EUI_2016,Group_min_EUI_2016,Group_max_EUI_2016,Group_std_EUI_2016,Group_size_EUI_2016,EER_km
1743,1014530043,52.9,202292.0,61.1,196400.0,58.3,196400.0,53.0,196400.0,Multifamily Housing,MN,17266.0,196400.0,2425.0,193975.0,2425.0,0.0,16.0,195.0,198.0,1928.0,996833.0,220838.0,10075.0,308 EAST 79 STREET,40.772817,-73.954570,89.0,57.433333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.249381
2502,4033410024,60.4,70000.0,65.6,63636.0,68.3,63636.0,67.9,63636.0,Multifamily Housing,QN,12875.0,63636.0,0.0,63636.0,0.0,0.0,6.0,54.0,54.0,1939.0,1029952.0,200442.0,11375.0,114-06 QUEENS BOULEVARD,40.716726,-73.835136,78.0,64.766667,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.319490
863,1015787501,67.9,101354.0,82.3,121052.0,86.0,121120.0,88.1,121120.0,Multifamily Housing,MN,4333.0,98881.0,3000.0,95881.0,3000.0,0.0,40.0,81.0,82.0,1988.0,998918.0,220664.0,10028.0,52 EAST END AVENUE,40.772336,-73.947043,29.0,78.733333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.414537
722,1004870012,42.0,74536.0,48.5,63888.0,46.4,63888.0,50.1,63888.0,Multifamily Housing,MN,11000.0,63888.0,10000.0,53888.0,0.0,10000.0,6.0,9.0,11.0,1900.0,983676.0,202936.0,10012.0,383 WEST BROADWAY,40.723689,-74.002071,117.0,45.633333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.235736
3684,2039387501,0.1,1749835.0,0.1,8512479.0,116.6,8512479.0,114.9,8512479.0,Multifamily Housing,BX,9273.0,2402218.0,437128.0,1965090.0,36444.0,400684.0,9.0,2659.0,2719.0,1941.0,1023285.0,243926.0,10462.0,62 PARKCHESTER ROAD,40.836110,-73.858934,76.0,38.933333,2,2944.470253,223.0,18.7,32513.5,7810.753742,158.0,0.039022
1023,1011257501,49.9,131200.0,53.4,141885.0,86.9,141885.0,96.3,141885.0,Multifamily Housing,MN,10217.0,100098.0,2485.0,97613.0,2485.0,0.0,16.5,125.0,128.0,1928.0,990347.0,222500.0,10023.0,41 WEST 72 STREET,40.777385,-73.977986,89.0,63.400000,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.453121
226,1005080025,105.2,85088.0,105.0,92088.0,111.1,92088.0,94.7,92088.0,Multifamily Housing,MN,15955.0,92088.0,5000.0,87088.0,0.0,0.0,10.0,162.0,163.0,1991.0,985928.0,203152.0,10012.0,65 EAST HOUSTON STREET,40.724282,-73.993946,26.0,107.100000,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.445592
3620,2036630002,30.4,90363.0,28.6,361452.0,23.7,361452.0,28.5,361452.0,Multifamily Housing,BX,350840.0,361452.0,0.0,361452.0,0.0,0.0,15.0,408.0,408.0,1958.0,1021444.0,239671.0,10473.0,1760 BRUCKNER BOULEVARD,40.824439,-73.865611,59.0,27.566667,0,265.500117,105.1,0.0,49924.0,2026.773072,855.0,0.107345
2859,2024610045,122.4,145728.0,123.9,145728.0,117.4,145728.0,108.9,145728.0,Multifamily Housing,BX,39250.0,145728.0,0.0,145728.0,0.0,0.0,6.0,108.0,108.0,1929.0,1006123.0,241220.0,10451.0,940 GRAND CONCOURSE,40.828742,-73.920963,88.0,121.233333,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.512408
7516,4097120151,118.0,51058.0,140.9,55722.0,156.9,55722.0,144.9,55722.0,Multifamily Housing,QN,27050.0,55722.0,0.0,55722.0,0.0,0.0,6.0,72.0,72.0,1952.0,1035475.0,198160.0,11435.0,141-40 84 DRIVE,40.710432,-73.815230,65.0,138.600000,1,212.526130,85.7,0.1,116258.9,2818.991617,6636.0,0.681799


# !!! @ BEN: INSERT YOUR LINEAR REGRESSION SCRIPT HERE !!!

input dataset: use model_df

output dataset: name it lr_df with on the following columns:
- BBL
- Pred_EER
- EER_lr = (EUI_2016/ Pred_EER) <- not needed based on our discussion, but let's just do it in case we need it

---

## Data Integration and Export

- Integrate datasets from K-Means and Linear Regression
- Compute Average EER of K-Means and Linear Regression
- Export CSV to the same folder

In [57]:
final_df = km_df.merge(lr_df, how = "left", left_on = "BBL", right_on = "BBL")

NameError: name 'lr_df' is not defined

In [58]:
final_df['avgEER'] = final_df['EUI_2016'] / ((final_df['Group_mean_EUI_2016'] + final_df['Pred_EER'])/2)

NameError: name 'final_df' is not defined

In [59]:
final_df.to_csv("final_df.csv")

NameError: name 'final_df' is not defined